### GPT-2: A Mathematical Overview

**Introduction**:
GPT-2 (Generative Pre-trained Transformer 2) is an advanced deep learning model designed for natural language processing tasks, specifically in generating coherent and contextually relevant text. It builds upon the transformer architecture, characterized by its utilization of self-attention mechanisms and feed-forward neural networks, to effectively capture the complexities and nuances of human language.

**1. Architectural Framework**:
At its core, GPT-2 employs the Transformer architecture, which consists of several key components:

- **Layers**: The model consists of a stack of multiple transformer blocks, each containing a multi-head self-attention mechanism and a feed-forward neural network.

- **Multi-Head Self-Attention**: This mechanism enables the model to assess the importance of different words in a sequence with respect to one another. For a given input, represented by embedding matrices \( X \), the multi-head attention is expressed mathematically as follows:
  
  $$
  \text{MultiHead}(X) = \text{Concat}(\text{head}_1, \ldots, \text{head}_h)W^O
  $$

  where each head is computed as:
  
  $$
  \text{head}_i = \text{Attention}(XW_i^Q, XW_i^K, XW_i^V)
  $$
  
  Here, \( W_i^Q, W_i^K, W_i^V \) are learnable projection matrices for queries, keys, and values. The attention function, applied to each head, is defined as:

  $$
  \text{Attention}(Q, K, V) = \text{softmax}\left( \frac{QK^T}{\sqrt{d_k}} \right) V
  $$

  where \( d_k \) is the dimensionality of the keys.

**2. Multilayer perceptron**:
Each transformer block includes a MLP block, which applies two linear transformations with a non-linear activation function (typically ReLU):

$$
\text{MLP}(x) = \text{max}(0, xW_1 + b_1)W_2 + b_2
$$

where \( W_1, W_2 \) are weight matrices, and \( b_1, b_2 \) are biases, facilitating complex transformations of the input.

**3. Loss Function and Training**:
GPT-2 utilizes a causal language modeling approach during training, wherein the objective is to predict the next word in a sequence given the preceding context. The model is trained using maximum likelihood estimation with a cross-entropy loss function:

$$
\mathcal{L} = -\sum_{t=1}^{n} \log P(w_t | w_1, w_2, \ldots, w_{t-1})
$$

where $P(w_t | w_1, w_2, \ldots, w_{t-1})$ denotes the probability of the next word $w_t$ conditioned on the previous words in the sequence.

**4. Pre-training and Fine-tuning**:
GPT-2 is pre-trained on a vast corpus of text using unsupervised learning techniques. This pre-training phase enables the model to derive context and language patterns effectively. The model can subsequently be fine-tuned on specific tasks or datasets to adapt its capabilities to particular applications, enhancing performance on downstream tasks such as text generation, summarization, or dialogue generation.


**In summary**, GPT-2 represents a significant advancement in the field of natural language processing, combining sophisticated mathematical constructs with deep learning techniques. Its architecture, characterized by self-attention mechanisms and feed-forward networks, allows it to generate human-like text based on contextual cues, making it a powerful tool for a variety of applications in language understanding and generation.

### 1. Environment variable setting block:

The following block is required to set environment variables that are read during the execution of the program code. 

User can change these environment variables between runs.

In [1]:
# Preliminary setup of experimental environment
import os
from pathlib import Path
import subprocess

nntile_dir = Path.cwd() / ".."

# Set environment variables
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Limit CUDA visibility
os.environ["OMP_NUM_THREADS"] = "1" # Disable BLAS parallelism
os.environ["PYTHONPATH"] = str(nntile_dir / "build" / "wrappers" / "python") # Path to a binary dir of NNTile Python wrappers

# All StarPU environment variables are available at https://files.inria.fr/starpu/doc/html/ExecutionConfigurationThroughEnvironmentVariables.html
os.environ["STARPU_NCPU"] = "1" # Use only 1 CPU core
os.environ["STARPU_NCUDA"] = "1" # Use only 1 CUDA device
os.environ["STARPU_SILENT"] = "1" # Do not show lots of StarPU outputs
os.environ["STARPU_SCHED"] = "dmdasd" # Name StarPU scheduler to be used
os.environ["STARPU_FXT_TRACE"] = "0" # Do not generate FXT traces
os.environ["STARPU_WORKERS_NOBIND"] = "1" # Do not bind workers (it helps if several instances of StarPU run in parallel)
os.environ["STARPU_PROFILING"] = "1" # This enables logging performance of workers and bandwidth of memory nodes
os.environ["STARPU_BUS_STATS"] = "1" # This enables logging of bus usage, prined at the end of execution
os.environ["STARPU_HOME"] = str(Path.cwd() / "starpu") # Main directory in which StarPU stores its configuration files
os.environ["STARPU_PERF_MODEL_DIR"] = str(Path(os.environ["STARPU_HOME"]) / "sampling") # Main directory in which StarPU stores its performance model files
os.environ["STARPU_PERF_MODEL_HOMOGENEOUS_CPU"] = "1" # Assume all CPU cores are equal
os.environ["STARPU_PERF_MODEL_HOMOGENEOUS_CUDA"] = "1" # Assume all CUDA devices are equal
os.environ["STARPU_HOSTNAME"] = "GPT2_LMHead_example" # Force the hostname to be used when managing performance model files
os.environ["STARPU_FXT_PREFIX"] = str(Path(os.environ["STARPU_HOME"]) / "fxt") # Directory to store FXT traces if enabled

# TODO: remove before merging PR
os.environ["LD_LIBRARY_PATH"] = "../../install/starpu-1.4-9b274af/lib"
os.environ["PYTHONPATH"] = str(nntile_dir / "build-1.4-9b274af" / "wrappers" / "python") # Path to a binary dir of NNTile Python wrappers

### 2. Data Preparation Block: 

This block uses the interpreted file "causal_lm_data_preparation.py". This Python script supports the following arguments when run:
- hf-dataset, (default=`"roneneldan/TinyStories"`): The name of the dataset to be processed and prepared for use in the training process. By default, the "TinyStories" dataset from the Huggingface infrastructure is specified,
- dataset-path, (default=`".data"`): path to the directory where previously downloaded datasets from remote sources are saved, making it easy to access for the future use,
- dataset-select, (`int`, default=`100`): specifies the number of records from the original dataset that fall into the training set,
- hf-tokenizer, (`str`, default=`"kimihailv/llama-1.3b"`): specifies the repository from the Huggingface infrastructure used as a tokenizer,
- tokenizer-path, (`str`, default=`".model"`): path to the directory where previously downloaded tokenizers are saved,
- seq-len, (`int`, default=`1024`): length of the input token sequence for the training process,
- batch-size, (`int`, default=`1`): batch size for the training process, then is the number of input data sentences between which the loss function optimizer step is called.

In [2]:
# Prepare TinyStories dataset into train.bin file
!python ../wrappers/python/examples/causal_lm_data_preparation.py \
    --hf-tokenizer="openai-community/gpt2" --seq-len=512 --batch-size=256 --dataset-select=5000

Token indices sequence length is longer than the specified maximum sequence length for this model (1106 > 1024). Running this sequence through the model will result in indexing errors


### 3. Example Scenarios

Below we show an example of utilizing the GPT-2 model, implemented using the NNTile framework. We explore the following scenarios:

- **Training the model from a random initial state and saving it to a checkpoint.**
- **Loading the model weights from a checkpoint and continuing training with a different data type.**
- **Training the remote model downloaded from the Hugging Face infrastructure.**
- **Performing inference with the pre-trained model given input prompt.**

For training and continuing retraining scenarios, the interpreted file "gpt2_lmhead_training.py" is used. This Python script supports the following arguments when running:

- remote_model_name, (`str`, default=`"openai-community/gpt2"`): This parameter specifies the name of the GPT-2 based model that resides within the HuggingFace infrastructure and will be utilized to initialize the configuration and the intial state of the NNTile model.

- pretrained, (choices=`["local", "remote"]`, default=`"local"`): This flag indicates the location of the pretrained model, with the `local` option requiring a configuration path (`config-path`) to start training from a randomly initialized state unless the checkpoint (`checkpoint-path`) is provided, in which case training continues from the last saved checkpoint state.

- checkpoint-path, (`str`, default=`""`): This refers to the file path where a saved checkpoint can be found, allowing for the resumption of training from a specific point if available.

- config-path, (`str`, default=`""`): This denotes the path to the configuration .json file that must be provided in the current version if the `pretrained` parameter is set to `"local"`.

- save-checkpoint-path, (`str`, default=`".model"`): This parameter specifies the directory path where intermediate checkpoints will be saved during the training process for future reference.

- optimizer, (choices=`["sgd", "adam", "adamw"]`, default=`"adam"`): This defines the type of optimizer that will be employed during the training process; the current version of NNTile supports three distinct optimization methods.

- model-path, (`str`, default=`".model"`): This indicates the directory path where previously loaded remote models are stored, facilitating easy access for further use.

- seq-len, (`int`, default=`1024`): length of the input token sequence for training.

- seq-len-tile, (`int`, default=`1024`): split size of sequence length into tiles

- batch-size, (`int`, default=`1`): batch size for the training process, which specifies the number of sentences processed by seq-len tokens between steps of the loss function optimizer.

- minibatch-size, (`int`, default=`-1`): batch size for which memory is allocated during training. The entire batch is divided into whole minibatches. All minibatches from one batch are fed through the model one by one to accumulate parameter gradients.

- minibatch-size-tile, (`int`, default=`-1`): batch size that goes to the CPU or GPU for calculations. Each minibatch must be divisible by an integer number of minibatch tiles.

- hidden-size-tile, (`int`, default=`-1`): the size of the pieces (tiles) into which the "hidden size" dimension is divided (also known as "embedding size") – the size of the multidimensional space into which incoming tokens are mapped. Only "piecewise" tensors of size hidden-size-tile along the corresponding axis are processed on the CPU and GPU.

- intermediate-size-tile, (`int`, default=`-1`): the size of the pieces (tiles) into which the "intermediate size" dimension is divided. Only "piecewise" tensors of size intermediate-size-tile along the corresponding axis are processed on the CPU and GPU.

- n-head-tile, (`int`, default=`-1`): the size of the pieces (tiles) into which the number of heads of the transformer layer is divided. Only “piecewise” tensors with a size of n-head-tile along the corresponding axis are processed by the CPU and GPU.

- dtype, (choices=`["fp32", "fp64", "tf32", "bf16", "fp32_fast_fp16", "fp32_fast_bf16"]`, default=`"fp32"`): This parameter outlines the various data types supported by NNTile, allowing users the flexibility to choose based on their model requirements.

- restrict, (choices=`["cpu", "cuda", None]`, default=`None`): This option allows users to specify restrictions on the computational resources utilized during training; selecting `"cpu"` restricts training to CPU-only cores, `"cuda"` limits it to GPU cores, while setting it to None allows for training across all available cores.

- flash-attention, (action=`"store_true"`): a boolean flag that, when used in the argument string, enables the current implementation of the Flash Attention algorithm (low-level Flash Attention kernels are currently not available) to process data in the attention block of Transformers neural networks.

- use-redux, (action=`"store_true"`): a boolean flag that, when used in the argument string, allows for the computation of dependent tasks simultaneously, with the subsequent reduction of the results into a single tensor.

- dataset-path, (default=`".data"`): path to the directory where previously prepared datasets are saved.

- dataset-file, (default=`""`): path (relative to dataset-path) to the .bin file that is created in the block of data preparation for training.

- lr, (`float`, default=`1e-4`): step length for the optimization algorithm.

- nepochs, (`int`, default=`1`): number of full passes through the training set.

- logger, (action=`"store_true"`): a boolean flag that enables special logger thread, that forwards logs to a provided remote logger server

- logger-server-addr, (default=`"localhost"`): remote logger server URL

- logger-server-port, (`int`, default=`5001`): remote logger server port

- ooc, (action=`"store_true"`): a boolean flag that enables Out-of-Core support to offload data from CPU RAM to path on a hard drive.

- ooc-path, (default=`"/tmp/nntile_ooc"`): a path on hard drive, where all the temporary buffers shall be stored.

- ooc-size, (`int`, default=`1073741824`): number of bytes, that limits temporary buffers size on the hard drive.

- force-offload-disk-portion-parameters, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of parameters to have Out-of-Core support enabled. All the rest parameters will have their Out-of-Core support disabled.

- force-offload-disk-portion-gradients, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of gradients to have Out-of-Core support enabled. All the rest gradients will have their Out-of-Core support disabled.

- force-offload-disk-portion-activations, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of inter-layer activations to have Out-of-Core support enabled. All the rest inter-layer activations will have their Out-of-Core support disabled.

- force-offload-disk-portion-temporaries, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of temporary buffers to have Out-of-Core support enabled. All the rest temporary buffers will have their Out-of-Core support disabled.

- force-offload-disk-portion-optimizer, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of optimizer states to have Out-of-Core support enabled. All the rest parameter states will have their Out-of-Core support disabled.

- force-offload-ram-portion-parameters, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of parameters to be forced for offloading. All the rest parameters will not be offloaded in advance at all.

- force-offload-ram-portion-gradients, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of gradients to be forced for offloading. All the rest gradients will not be offloaded in advance at all.

- force-offload-ram-portion-activations, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of inter-layer activations to be forced for offloading. All the rest inter-layer activations will not be offloaded in advance at all.

- force-offload-ram-portion-temporaries, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of temporary buffers to be forced for offloading. All the rest temporary buffers will not be offloaded in advance at all.

- force-offload-ram-portion-optimizer, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of optimizer states to be forced for offloading. All the rest parameter states will not be offloaded in advance at all.

#### 3.1. Training from the random initial state and saving into checkpoint.

This requires option `pretrained` set to `local` and `config-path` to point on previously created `.json` configuration file.

In this example, we start training in the fp32_fast_fp16 type.

In [3]:
# Launch an external python process to train gpt2_lmhead model on TinyStories
!python ../wrappers/python/examples/gpt2_lmhead_training.py \
    --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt2_default_config.json" \
    --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=fp32_fast_fp16 --nepochs=1 \
    --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin"

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='fp32_fast_fp16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=False, ooc_path='/tmp/nntile_ooc', ooc_size=1073741824, force_offload_disk_portion_parameters=0.0, force_offload_disk_portion_gradients=0.0, force_offload_disk_portion_activations=0.0, force_offload_disk_portion_temporaries=0.0, force_offload_disk_portion_optimizer=0.0, force_offload_ram_portion_parameters=0.0, force_offload_ram_portion_gradients=0.0, for

#### 3.2. Resume training from the local checkpoint.

This requires option `pretrained` again to be set to `local`, `config-path` to point on previously created `.json` configuration file, and also `checkpoint-path` to point on the pre-existing checkpoint file in the PyTorch format.

Training process can be resumed using a different data type and on a different set of compute nodes. For example, here we switch to the TF32 type (synonymous with the fp32_fast_tf32 type) and restrict to using only GPUs.

In [4]:
# Launch an external python process to finetune a pretrained NNTile gpt2_lmhead model on TinyStories
!python ../wrappers/python/examples/gpt2_lmhead_training.py \
    --restrict="cuda" --pretrained=local --checkpoint-path=".model/nntile_checkpoint.pt" \
    --config-path="../wrappers/python/examples/gpt2_default_config.json" \
    --save-checkpoint-path=".model/nntile_further_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=tf32 \
    --restrict="cuda" --nepochs=1 --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin"

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='.model/nntile_checkpoint.pt', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_further_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='tf32', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=False, ooc_path='/tmp/nntile_ooc', ooc_size=1073741824, force_offload_disk_portion_parameters=0.0, force_offload_disk_portion_gradients=0.0, force_offload_disk_portion_activations=0.0, force_offload_disk_portion_temporaries=0.0, force_offload_disk_portion_optimizer=0.0, force_offload_ram_portion_parameters=0.0, force_offload_ram_p

#### 3.3. Training from remote and saving into checkpoint.

Our framework currently supports the continuation of model training obtained from a remote source, as we show here with the Hugging Face library. The weights from the loaded model are transferred into the model implemented in NNTile. Consequently, training can be further advanced using any data type and across any set of computing nodes that accommodate the selected data type.

This requires option `pretrained` to be set to `remote`. Options `config-path` and `checkpoint-path` are no longer needed since model config is obtained from the remote model as well as layers' weights. Training can be resumed using any data type and on any set of compute nodes that support the selected data type. In the example below, we switch to the BF16 type here.

In [5]:
# Launch an external python process to finetune a downloaded from remote source pretrained gpt2 model on TinyStories
!python ../wrappers/python/examples/gpt2_lmhead_training.py \
    --restrict="cuda" --pretrained=remote --save-checkpoint-path=".model/nntile_remote_checkpoint.pt"\
    --optimizer="adam" --lr=1e-4 --dtype=bf16 --nepochs=1 --batch-size=256 --minibatch-size=8 \
    --dataset-file="tinystories/train.bin"

Namespace(remote_model_name='openai-community/gpt2', pretrained='remote', checkpoint_path='', config_path='', save_checkpoint_path='.model/nntile_remote_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='bf16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=False, ooc_path='/tmp/nntile_ooc', ooc_size=1073741824, force_offload_disk_portion_parameters=0.0, force_offload_disk_portion_gradients=0.0, force_offload_disk_portion_activations=0.0, force_offload_disk_portion_temporaries=0.0, force_offload_disk_portion_optimizer=0.0, force_offload_ram_portion_parameters=0.0, force_offload_ram_portion_gradients=0.0, force_offload_ram_portion_activations=0.0, force_offload_

### 4. Inference process.

In the current version of the GPT2 scenario, the NNTile framework model is created from a (pre-)loaded pre-trained GPT2 model from the Huggingface library. The model layer weights are passed to the corresponding NNTile model layers, and then the inference process is performed solely by NNTile, without any involvement of third-party models and mechanisms. To perform the inference, we use another program file - "gpt2_generate.py". The program code supports the following arguments when running:

- cache_dir, (`str`, default="cache_hf"): path to the directory where previously loaded models from a remote source are saved,
- max-seq-len, (`int`, default=1024): maximum length of the input token sequence,
- model, (`str`, default=`"gpt2"`): name of the model from the HuggingFace framework that will be used to initialize the configuration and initial state of the NNTile model,
- restrict, (choices=`["cpu", "cuda", None]`, default=`None`): limit on the computing resources used during inference; `"cpu"` restricts inference to CPU cores only, `"cuda"` - to GPU cores only, while None allows using all available cores,
- prompt, (`str`, default=`"What do you think about dogs?"`): input query, a string fed to the model input to perform inference based on it,
- generation-mode, (choices = `["Greedy", "TopK", "TopP"]`, default=`"Greedy"`): token generation mode in the GenerationMode class object (described in the "llm_params.py" file),
- parallel-sampling-mode, (choices=`["BeamSearch", "Parallel"]`, default=`"BeamSearch"`): parallel generation mode for multiple responses to a single query in the ParallelSamplingMode class object (described in the "llm_params.py" file),
- max-tokens, (`int`, default=`100`): maximum number of generated tokens, including user request tokens,
- use-cache, (action=`"store_true"`): boolean flag, when used in the argument line, enables the use of KV caches, allowing to reuse previously calculated values,
- top-k, (`int`, default=`None`): probabilistic selection based on the top-k most probable tokens,
- top-p-thr, (`float`, default=`None`): probabilistic selection based on tokens whose probability is not lower than the top-p-thr threshold,
- temperature, (`float`, default=`1.0`): "temperature" parameter for token generation,
- num-beams, (`int`, default=`1`): number of beams for parallel generation mode.

#### 4.1. Examples with different types of generation strategies

`BeamSearch` generation strategy and number of beams set to the default value of `1`.

In [6]:
!python ../wrappers/python/examples/gpt2_generate.py --cache_dir=.model --max-seq-len=512 \
    --model=gpt2 --restrict=cuda --use-cache \
    --prompt="Why does the Sun shine?" \
    --max-tokens=20

Notice:
 None
Why does the Sun shine?

The Sun is the most visible star in the sky. It

#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	0.6083 GB	192.4658 MB/s	(transfers : 229 - avg 2.7203 MB)
	CUDA 0 -> NUMA 0	0.0036 GB	1.1255 MB/s	(transfers : 16 - avg 0.2277 MB)
Total transfers: 0.6119 GB
#---------------------


`Parallel` generation strategy and number of beams set to `3`.

In [7]:
!python ../wrappers/python/examples/gpt2_generate.py --cache_dir=.model --max-seq-len=512 \
    --model=gpt2 --restrict=cuda --use-cache \
    --num-beams=3 --parallel-sampling-mode=Parallel \
    --prompt="Why does the Sun shine?" \
    --max-tokens=20

Notice:
 None
['Why does the Sun shine? The Sun is the most visible star in the sky, and it is', "Why does the Sun shine? It's a very important thing to know.\n\nThe Sun is", 'Why does the Sun shine?\n\nThe Sun is the most visible star in the sky. It']

#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	0.6083 GB	165.0282 MB/s	(transfers : 280 - avg 2.2248 MB)
	CUDA 0 -> NUMA 0	0.0084 GB	2.2856 MB/s	(transfers : 42 - avg 0.2054 MB)
Total transfers: 0.6168 GB
#---------------------


`BeamSearch` generation strategy and number of beams set to `3`.

In [8]:
!python ../wrappers/python/examples/gpt2_generate.py --cache_dir=.model --max-seq-len=512 \
    --model=gpt2 --restrict=cuda --use-cache \
    --num-beams=3 --parallel-sampling-mode=BeamSearch \
    --prompt="Why does the Sun shine?" \
    --max-tokens=20

Notice:
 None
['Why does the Sun shine?\n\nThe Sun is the brightest star in the sky, and it', "Why does the Sun shine?\n\nThe Sun is the brightest star in the sky. It's", 'Why does the Sun shine?\n\nThe Sun is the brightest star in the sky. It is']

#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	0.6083 GB	190.1880 MB/s	(transfers : 280 - avg 2.2248 MB)
	CUDA 0 -> NUMA 0	0.0084 GB	2.6340 MB/s	(transfers : 42 - avg 0.2054 MB)
Total transfers: 0.6168 GB
#---------------------


#### 4.2. Examples with different token generation modes and temperatures

`TopK` token generation strategy with default temperature value of `1.0`.

In [9]:
!python ../wrappers/python/examples/gpt2_generate.py --cache_dir=.model --max-seq-len=512 \
    --model=gpt2 --restrict=cuda --use-cache \
    --generation-mode=TopK --top-k=10 \
    --prompt="Why does the Sun shine?" \
    --max-tokens=20

Notice:
 None
Why does the Sun shine?

The Sun shines because of its light. The sun doesn't

#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	0.6083 GB	229.5236 MB/s	(transfers : 229 - avg 2.7203 MB)
	CUDA 0 -> NUMA 0	0.0036 GB	1.3422 MB/s	(transfers : 16 - avg 0.2277 MB)
Total transfers: 0.6119 GB
#---------------------


`TopK` token generation strategy with the temperature value of `100.0`.

In [10]:
!python ../wrappers/python/examples/gpt2_generate.py --cache_dir=.model --max-seq-len=512 \
    --model=gpt2 --restrict=cuda --use-cache \
    --generation-mode=TopK --top-k=10 --temperature=100.0 \
    --prompt="Why does the Sun shine?" \
    --max-tokens=20

Notice:
 None
Why does the Sun shine? The moon shines from above. But is its brightness actually due directly or

#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	0.6083 GB	225.4867 MB/s	(transfers : 229 - avg 2.7203 MB)
	CUDA 0 -> NUMA 0	0.0036 GB	1.3186 MB/s	(transfers : 16 - avg 0.2277 MB)
Total transfers: 0.6119 GB
#---------------------


`TopK` token generation strategy with the temperature value of `0.01`.

In [11]:
!python ../wrappers/python/examples/gpt2_generate.py --cache_dir=.model --max-seq-len=512 \
    --model=gpt2 --restrict=cuda --use-cache \
    --generation-mode=TopK --top-k=10 --temperature=0.01 \
    --prompt="Why does the Sun shine?" \
    --max-tokens=20

Notice:
 None
Why does the Sun shine?

The Sun is the most visible star in the sky. It

#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	0.6083 GB	215.2783 MB/s	(transfers : 229 - avg 2.7203 MB)
	CUDA 0 -> NUMA 0	0.0036 GB	1.2589 MB/s	(transfers : 16 - avg 0.2277 MB)
Total transfers: 0.6119 GB
#---------------------


`TopP` token generation strategy with default temperature value of `1.0`.

In [12]:
!python ../wrappers/python/examples/gpt2_generate.py --cache_dir=.model --max-seq-len=512 \
    --model=gpt2 --restrict=cuda --use-cache \
    --generation-mode=TopP --top-p=0.1 \
    --prompt="Why does the Sun shine?" \
    --max-tokens=20

Notice:
 None
Why does the Sun shine? Solvent of evil in matter, the consummation of righteousness,

#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	0.6083 GB	238.5161 MB/s	(transfers : 229 - avg 2.7203 MB)
	CUDA 0 -> NUMA 0	0.0036 GB	1.3948 MB/s	(transfers : 16 - avg 0.2277 MB)
Total transfers: 0.6119 GB
#---------------------


`TopP` token generation strategy with the temperature value of `100.0`.

In [13]:
!python ../wrappers/python/examples/gpt2_generate.py --cache_dir=.model --max-seq-len=512 \
    --model=gpt2 --restrict=cuda --use-cache \
    --generation-mode=TopP --top-p=0.1 --temperature=100.0 \
    --prompt="Why does the Sun shine?" \
    --max-tokens=20

Notice:
 None
Why does the Sun shine? Eleiblume grounds underground crackingtons living trapped pellets./Panel Triaura

#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	0.6083 GB	228.0702 MB/s	(transfers : 229 - avg 2.7203 MB)
	CUDA 0 -> NUMA 0	0.0036 GB	1.3337 MB/s	(transfers : 16 - avg 0.2277 MB)
Total transfers: 0.6119 GB
#---------------------


`TopP` token generation strategy with the temperature value of `0.01`.

In [14]:
!python ../wrappers/python/examples/gpt2_generate.py --cache_dir=.model --max-seq-len=512 \
    --model=gpt2 --restrict=cuda --use-cache \
    --generation-mode=TopP --top-p=0.1 --temperature=0.01 \
    --prompt="Why does the Sun shine?" \
    --max-tokens=20

Notice:
 None
Why does the Sun shine?

The Sun is the most visible star in the sky. It

#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	0.6083 GB	225.2096 MB/s	(transfers : 229 - avg 2.7203 MB)
	CUDA 0 -> NUMA 0	0.0036 GB	1.3169 MB/s	(transfers : 16 - avg 0.2277 MB)
Total transfers: 0.6119 GB
#---------------------


### 5. When GPU VRAM is not enough: offloading data from GPU VRAM to CPU RAM in advance
Unfortunately, current state of NNTile does not automatically define which data shall be offloaded from GPU VRAM to CPU RAM. Instead, a user shall mark each NNTile tensor as either "forced to be offloaded in advance to RAM after each update" or "not offloaded in advance at all". For simplicity, the following flags control this option:

- force-offload-ram-portion-parameters, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of parameters to be forced for offloading. All the rest parameters will not be offloaded in advance at all.

- force-offload-ram-portion-gradients, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of gradients to be forced for offloading. All the rest gradients will not be offloaded in advance at all.

- force-offload-ram-portion-activations, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of inter-layer activations to be forced for offloading. All the rest inter-layer activations will not be offloaded in advance at all.

- force-offload-ram-portion-temporaries, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of temporary buffers to be forced for offloading. All the rest temporary buffers will not be offloaded in advance at all.

- force-offload-ram-portion-optimizer, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of optimizer states to be forced for offloading. All the rest parameter states will not be offloaded in advance at all.



#### 5.1 Training experiment with no enforced data offloading from GPU VRAM to CPU RAM
Amount of data transfers is the following:
```sh
#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	0.0000 GB	0.0000 MB/s	(transfers : 0 - avg -nan MB)
	CUDA 0 -> NUMA 0	0.0000 GB	0.0000 MB/s	(transfers : 3 - avg 0.0000 MB)
Total transfers: 0.0000 GB
#---------------------
NNTile training time: 18.380743980407715 seconds
```

In [15]:
# Launch an external python process to train gpt2_lmhead model on TinyStories
!python ../wrappers/python/examples/gpt2_lmhead_training.py \
    --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt2_default_config.json" \
    --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=fp32_fast_fp16 --nepochs=1 \
    --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin" \
    --force-offload-ram-portion-parameters=0.0 --force-offload-ram-portion-gradients=0.0 \
    --force-offload-ram-portion-activations=0.0 --force-offload-ram-portion-temporaries=0.0 \
    --force-offload-ram-portion-optimizer=0.0

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='fp32_fast_fp16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=False, ooc_path='/tmp/nntile_ooc', ooc_size=1073741824, force_offload_disk_portion_parameters=0.0, force_offload_disk_portion_gradients=0.0, force_offload_disk_portion_activations=0.0, force_offload_disk_portion_temporaries=0.0, force_offload_disk_portion_optimizer=0.0, force_offload_ram_portion_parameters=0.0, force_offload_ram_portion_gradients=0.0, for

#### 5.2 Training experiment with enforced GPU->CPU offloading after each update for all parameters
The experiment is done with `--force-offload-ram-portion-parameters=1.0` option. Amount of data transfers is the following:
```sh
#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	5.7032 GB	309.9836 MB/s	(transfers : 1918 - avg 3.0449 MB)
	CUDA 0 -> NUMA 0	2.5210 GB	137.0249 MB/s	(transfers : 819 - avg 3.1521 MB)
Total transfers: 8.2242 GB
#---------------------
NNTile training time: 18.89538550376892 seconds
```

In [16]:
# Launch an external python process to train gpt2_lmhead model on TinyStories
!python ../wrappers/python/examples/gpt2_lmhead_training.py \
    --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt2_default_config.json" \
    --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=fp32_fast_fp16 --nepochs=1 \
    --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin" \
    --force-offload-ram-portion-parameters=1.0 --force-offload-ram-portion-gradients=0.0 \
    --force-offload-ram-portion-activations=0.0 --force-offload-ram-portion-temporaries=0.0 \
    --force-offload-ram-portion-optimizer=0.0

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='fp32_fast_fp16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=False, ooc_path='/tmp/nntile_ooc', ooc_size=1073741824, force_offload_disk_portion_parameters=0.0, force_offload_disk_portion_gradients=0.0, force_offload_disk_portion_activations=0.0, force_offload_disk_portion_temporaries=0.0, force_offload_disk_portion_optimizer=0.0, force_offload_ram_portion_parameters=1.0, force_offload_ram_portion_gradients=0.0, for

#### 5.3 Training experiment with enforced GPU->CPU offloading after each update for all optimizer states
The experiment is done with `--force-offload-ram-portion-optimizer=1.0` option. Amount of data transfers is the following:
```sh
#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	1.8280 GB	99.4796 MB/s	(transfers : 595 - avg 3.1459 MB)
	CUDA 0 -> NUMA 0	3.4945 GB	190.1761 MB/s	(transfers : 1182 - avg 3.0274 MB)
Total transfers: 5.3225 GB
#---------------------
NNTile training time: 18.86917495727539 seconds
```

In [17]:
# Launch an external python process to train gpt2_lmhead model on TinyStories
!python ../wrappers/python/examples/gpt2_lmhead_training.py \
    --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt2_default_config.json" \
    --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=fp32_fast_fp16 --nepochs=1 \
    --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin" \
    --force-offload-ram-portion-parameters=0.0 --force-offload-ram-portion-gradients=0.0 \
    --force-offload-ram-portion-activations=0.0 --force-offload-ram-portion-temporaries=0.0 \
    --force-offload-ram-portion-optimizer=1.0

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='fp32_fast_fp16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=False, ooc_path='/tmp/nntile_ooc', ooc_size=1073741824, force_offload_disk_portion_parameters=0.0, force_offload_disk_portion_gradients=0.0, force_offload_disk_portion_activations=0.0, force_offload_disk_portion_temporaries=0.0, force_offload_disk_portion_optimizer=0.0, force_offload_ram_portion_parameters=0.0, force_offload_ram_portion_gradients=0.0, for

#### 5.4 Training experiment with enforced GPU->CPU offloading after each update for all gradients
The experiment is done with `--force-offload-ram-portion-gradients=1.0` option. Amount of data transfers is the following:
```sh
#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	4.0997 GB	224.6789 MB/s	(transfers : 1466 - avg 2.8636 MB)
	CUDA 0 -> NUMA 0	60.4339 GB	3312.0267 MB/s	(transfers : 19532 - avg 3.1684 MB)
Total transfers: 64.5336 GB
#---------------------
NNTile training time: 18.74090600013733 seconds
```

In [18]:
# Launch an external python process to train gpt2_lmhead model on TinyStories
!python ../wrappers/python/examples/gpt2_lmhead_training.py \
    --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt2_default_config.json" \
    --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=fp32_fast_fp16 --nepochs=1 \
    --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin" \
    --force-offload-ram-portion-parameters=0.0 --force-offload-ram-portion-gradients=1.0 \
    --force-offload-ram-portion-activations=0.0 --force-offload-ram-portion-temporaries=0.0 \
    --force-offload-ram-portion-optimizer=0.0

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='fp32_fast_fp16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=False, ooc_path='/tmp/nntile_ooc', ooc_size=1073741824, force_offload_disk_portion_parameters=0.0, force_offload_disk_portion_gradients=0.0, force_offload_disk_portion_activations=0.0, force_offload_disk_portion_temporaries=0.0, force_offload_disk_portion_optimizer=0.0, force_offload_ram_portion_parameters=0.0, force_offload_ram_portion_gradients=1.0, for

#### 5.5 Training experiment with enforced GPU->CPU offloading after each update for 10% of inter-layer activations
The experiment is done with `--force-offload-ram-portion-activations=0.1` option. Amount of data transfers is the following:
```sh
#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	2.3295 GB	110.5767 MB/s	(transfers : 91 - avg 26.2131 MB)
	CUDA 0 -> NUMA 0	98.2119 GB	4661.9478 MB/s	(transfers : 3559 - avg 28.2577 MB)
Total transfers: 100.5414 GB
#---------------------
NNTile training time: 21.632005214691162 seconds
```

In [19]:
# Launch an external python process to train gpt2_lmhead model on TinyStories
!python ../wrappers/python/examples/gpt2_lmhead_training.py \
    --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt2_default_config.json" \
    --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=fp32_fast_fp16 --nepochs=1 \
    --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin" \
    --force-offload-ram-portion-parameters=0.0 --force-offload-ram-portion-gradients=0.0 \
    --force-offload-ram-portion-activations=0.1 --force-offload-ram-portion-temporaries=0.0 \
    --force-offload-ram-portion-optimizer=0.0

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='fp32_fast_fp16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=False, ooc_path='/tmp/nntile_ooc', ooc_size=1073741824, force_offload_disk_portion_parameters=0.0, force_offload_disk_portion_gradients=0.0, force_offload_disk_portion_activations=0.0, force_offload_disk_portion_temporaries=0.0, force_offload_disk_portion_optimizer=0.0, force_offload_ram_portion_parameters=0.0, force_offload_ram_portion_gradients=0.0, for

#### 5.6 Training experiment with enforced GPU->CPU offloading after each update for 10% of intra-layer temporary buffers
The experiment is done with `--force-offload-ram-portion-temporaries=0.1` option. Amount of data transfers is the following:
```sh
#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	2.5618 GB	78.9200 MB/s	(transfers : 51 - avg 51.4375 MB)
	CUDA 0 -> NUMA 0	342.2207 GB	10542.4892 MB/s	(transfers : 6245 - avg 56.1143 MB)
Total transfers: 344.7826 GB
#---------------------
NNTile training time: 33.303049087524414 seconds
```

In [20]:
# Launch an external python process to train gpt2_lmhead model on TinyStories
!python ../wrappers/python/examples/gpt2_lmhead_training.py \
    --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt2_default_config.json" \
    --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=fp32_fast_fp16 --nepochs=1 \
    --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin" \
    --force-offload-ram-portion-parameters=0.0 --force-offload-ram-portion-gradients=0.0 \
    --force-offload-ram-portion-activations=0.0 --force-offload-ram-portion-temporaries=0.1 \
    --force-offload-ram-portion-optimizer=0.0

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='fp32_fast_fp16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=False, ooc_path='/tmp/nntile_ooc', ooc_size=1073741824, force_offload_disk_portion_parameters=0.0, force_offload_disk_portion_gradients=0.0, force_offload_disk_portion_activations=0.0, force_offload_disk_portion_temporaries=0.0, force_offload_disk_portion_optimizer=0.0, force_offload_ram_portion_parameters=0.0, force_offload_ram_portion_gradients=0.0, for

#### 5.7.1 Training experiment with limited GPU memory without forced offloading in advance
GPU memory is limited to 19000 MB. Training time and data transfers are reported below:
```sh
#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	6.2240 GB	308.5422 MB/s	(transfers : 1118 - avg 5.7007 MB)
	CUDA 0 -> NUMA 0	12.3630 GB	612.8718 MB/s	(transfers : 1195 - avg 10.5939 MB)
Total transfers: 18.5870 GB
#---------------------
NNTile training time: 20.72080135345459 seconds
```

In [21]:
# Launch an external python process to train gpt2_lmhead model on TinyStories
!STARPU_LIMIT_CUDA_MEM=19000 python ../wrappers/python/examples/gpt2_lmhead_training.py \
    --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt2_default_config.json" \
    --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=fp32_fast_fp16 --nepochs=1 \
    --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin" \
    --force-offload-ram-portion-parameters=0.0 --force-offload-ram-portion-gradients=0.0 \
    --force-offload-ram-portion-activations=0.0 --force-offload-ram-portion-temporaries=0.0 \
    --force-offload-ram-portion-optimizer=0.0

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='fp32_fast_fp16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=False, ooc_path='/tmp/nntile_ooc', ooc_size=1073741824, force_offload_disk_portion_parameters=0.0, force_offload_disk_portion_gradients=0.0, force_offload_disk_portion_activations=0.0, force_offload_disk_portion_temporaries=0.0, force_offload_disk_portion_optimizer=0.0, force_offload_ram_portion_parameters=0.0, force_offload_ram_portion_gradients=0.0, for

#### 5.7.2 Training experiment with limited GPU memory with forced offloading in advance
GPU memory is limited to 19000 MB. This experiment only enables forced offloading in advance of parameters. Training time and data transfers are reported below:
```sh
#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	3.0767 GB	155.8677 MB/s	(transfers : 758 - avg 4.1563 MB)
	CUDA 0 -> NUMA 0	6.2408 GB	316.1688 MB/s	(transfers : 1445 - avg 4.4226 MB)
Total transfers: 9.3175 GB
#---------------------
NNTile training time: 20.26873540878296 seconds
```

In [22]:
# Launch an external python process to train gpt2_lmhead model on TinyStories
!STARPU_LIMIT_CUDA_MEM=19000 python ../wrappers/python/examples/gpt2_lmhead_training.py \
    --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt2_default_config.json" \
    --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=fp32_fast_fp16 --nepochs=1 \
    --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin" \
    --force-offload-ram-portion-parameters=0.0 --force-offload-ram-portion-gradients=0.0 \
    --force-offload-ram-portion-activations=0.0 --force-offload-ram-portion-temporaries=0.0 \
    --force-offload-ram-portion-optimizer=1.0

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='fp32_fast_fp16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=False, ooc_path='/tmp/nntile_ooc', ooc_size=1073741824, force_offload_disk_portion_parameters=0.0, force_offload_disk_portion_gradients=0.0, force_offload_disk_portion_activations=0.0, force_offload_disk_portion_temporaries=0.0, force_offload_disk_portion_optimizer=0.0, force_offload_ram_portion_parameters=0.0, force_offload_ram_portion_gradients=0.0, for

### 6. Training with Out-of-Core support (when there is not enough memory within CPU RAM)

To train with OOC (Out-of-Core) support, parameters `--ooc`, `--ooc-path`, `--ooc-force-portion-parameters`, `--ooc-force-portion-gradients`, `--ooc-force-portion-activations`, `--ooc-force-portion-temporaries` and `--ooc-force-portion-optimizer`. These parameters have the following meaning:
- ooc, (action=`"store_true"`): a boolean flag that enables Out-of-Core support to offload data from CPU RAM to path on a hard drive.

- ooc-path, (default=`"/tmp/nntile_ooc"`): a path on hard drive, where all the temporary buffers shall be stored.

- ooc-size, (`int`, default=`1073741824`): number of bytes, that limits temporary buffers size on the hard drive.

- ooc-force-portion-parameters, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of parameters to have Out-of-Core support enabled. All the rest parameters will have their Out-of-Core support disabled.

- ooc-force-portion-gradients, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of gradients to have Out-of-Core support enabled. All the rest gradients will have their Out-of-Core support disabled.

- ooc-force-portion-activations, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of inter-layer activations to have Out-of-Core support enabled. All the rest inter-layer activations will have their Out-of-Core support disabled.

- ooc-force-portion-temporaries, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of temporary buffers to have Out-of-Core support enabled. All the rest temporary buffers will have their Out-of-Core support disabled.

- ooc-force-portion-optimizer, (`float`, default=`0.0`): a float value between 0.0 and 1.0, indicating a portion of optimizer states to have Out-of-Core support enabled. All the rest parameter states will have their Out-of-Core support disabled.

#### 6.1. Train with OOC support enabled, but no actual offloading to Disk happens. 

GPU memory is limited to 19000 MB. This is a baseline experiment with no offloading to the Disk. Before the training procedure bandwidth of Disk is measured:
```sh
#---------------------
Data transfer speed for /tmp/nntile_ooc (node 2):
0 -> 2: 1353 MB/s
1 -> 2: 1284 MB/s
2 -> 0: 1353 MB/s
2 -> 1: 1282 MB/s
0 -> 2: 29 us
1 -> 2: 39 us
2 -> 0: 29 us
2 -> 1: 39 us

#---------------------
```
Memory nodes:
- `0`: CPU RAM
- `1`: GPU VRAM
- `2`: Hard drive

Training time and data transfers are reported below:
```sh
#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	6.2240 GB	306.9697 MB/s	(transfers : 1118 - avg 5.7007 MB)
	CUDA 0 -> NUMA 0	12.3630 GB	609.7482 MB/s	(transfers : 1195 - avg 10.5939 MB)
	Disk 0 -> NUMA 0	0.0000 GB	0.0000 MB/s	(transfers : 0 - avg -nan MB)
	NUMA 0 -> Disk 0	0.0000 GB	0.0000 MB/s	(transfers : 0 - avg -nan MB)
Total transfers: 18.5870 GB
#---------------------
NNTile training time: 20.816964864730835 seconds
```

In [23]:
# Launch an external python process to train gpt2_lmhead model on TinyStories
!STARPU_LIMIT_CUDA_MEM=19000 python ../wrappers/python/examples/gpt2_lmhead_training.py \
    --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt2_default_config.json" \
    --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=fp32_fast_fp16 --nepochs=1 \
    --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin" \
    --ooc --ooc-path="/tmp/nntile_ooc" --ooc-size=10737418240 \
    --force-offload-disk-portion-parameters=0.0 --force-offload-disk-portion-gradients=0.0 \
    --force-offload-disk-portion-activations=0.0 --force-offload-disk-portion-temporaries=0.0 \
    --force-offload-disk-portion-optimizer=0.0

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='fp32_fast_fp16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=True, ooc_path='/tmp/nntile_ooc', ooc_size=10737418240, force_offload_disk_portion_parameters=0.0, force_offload_disk_portion_gradients=0.0, force_offload_disk_portion_activations=0.0, force_offload_disk_portion_temporaries=0.0, force_offload_disk_portion_optimizer=0.0, force_offload_ram_portion_parameters=0.0, force_offload_ram_portion_gradients=0.0, for

#### 6.2. Train with OOC support enabled, only parameters are offloaded to Disk. 

GPU memory is limited to 19000 MB. Only parameters are offloaded to the Disk. Training time and data transfers are reported below:
```sh
#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	9.4190 GB	412.1612 MB/s	(transfers : 2313 - avg 4.1699 MB)
	CUDA 0 -> NUMA 0	15.0687 GB	659.3823 MB/s	(transfers : 2035 - avg 7.5825 MB)
	Disk 0 -> NUMA 0	3.1414 GB	137.4645 MB/s	(transfers : 1167 - avg 2.7565 MB)
	NUMA 0 -> Disk 0	2.3728 GB	103.8317 MB/s	(transfers : 807 - avg 3.0109 MB)
Total transfers: 30.0019 GB
#---------------------
NNTile training time: 23.450611352920532 seconds
```

In [24]:
# Launch an external python process to train gpt2_lmhead model on TinyStories
!STARPU_LIMIT_CUDA_MEM=19000 python ../wrappers/python/examples/gpt2_lmhead_training.py \
    --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt2_default_config.json" \
    --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=fp32_fast_fp16 --nepochs=1 \
    --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin" \
    --ooc --ooc-path="/tmp/nntile_ooc" --ooc-size=10737418240 \
    --force-offload-disk-portion-parameters=1.0 --force-offload-disk-portion-gradients=0.0 \
    --force-offload-disk-portion-activations=0.0 --force-offload-disk-portion-temporaries=0.0 \
    --force-offload-disk-portion-optimizer=0.0

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='fp32_fast_fp16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=True, ooc_path='/tmp/nntile_ooc', ooc_size=10737418240, force_offload_disk_portion_parameters=1.0, force_offload_disk_portion_gradients=0.0, force_offload_disk_portion_activations=0.0, force_offload_disk_portion_temporaries=0.0, force_offload_disk_portion_optimizer=0.0, force_offload_ram_portion_parameters=0.0, force_offload_ram_portion_gradients=0.0, for

#### 6.3. Train with OOC support enabled, only optimizer states are offloaded to Disk. 

GPU memory is limited to 19000 MB. Only optimizer states are offloaded to the Disk. Training time and data transfers are reported below:
```sh
#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	3.1474 GB	148.5624 MB/s	(transfers : 648 - avg 4.9737 MB)
	CUDA 0 -> NUMA 0	6.5052 GB	307.0538 MB/s	(transfers : 1459 - avg 4.5657 MB)
	Disk 0 -> NUMA 0	1.5683 GB	74.0257 MB/s	(transfers : 430 - avg 3.7348 MB)
	NUMA 0 -> Disk 0	2.9634 GB	139.8773 MB/s	(transfers : 930 - avg 3.2630 MB)
Total transfers: 14.1844 GB
#---------------------
NNTile training time: 21.748796939849854 seconds
```

In [25]:
# Launch an external python process to train gpt2_lmhead model on TinyStories
!STARPU_LIMIT_CUDA_MEM=19000 python ../wrappers/python/examples/gpt2_lmhead_training.py \
    --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt2_default_config.json" \
    --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=fp32_fast_fp16 --nepochs=1 \
    --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin" \
    --ooc --ooc-path="/tmp/nntile_ooc" --ooc-size=10737418240 \
    --force-offload-disk-portion-parameters=0.0 --force-offload-disk-portion-gradients=0.0 \
    --force-offload-disk-portion-activations=0.0 --force-offload-disk-portion-temporaries=0.0 \
    --force-offload-disk-portion-optimizer=1.0

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='fp32_fast_fp16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=True, ooc_path='/tmp/nntile_ooc', ooc_size=10737418240, force_offload_disk_portion_parameters=0.0, force_offload_disk_portion_gradients=0.0, force_offload_disk_portion_activations=0.0, force_offload_disk_portion_temporaries=0.0, force_offload_disk_portion_optimizer=1.0, force_offload_ram_portion_parameters=0.0, force_offload_ram_portion_gradients=0.0, for

#### 6.4. Train with OOC support enabled, only gradients are offloaded to Disk. 

GPU memory is limited to 19000 MB. Only gradients are offloaded to the Disk. Training time and data transfers are reported below:
```sh
#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	9.2661 GB	352.4163 MB/s	(transfers : 2246 - avg 4.2246 MB)
	CUDA 0 -> NUMA 0	68.9892 GB	2623.8428 MB/s	(transfers : 20584 - avg 3.4320 MB)
	Disk 0 -> NUMA 0	3.3038 GB	125.6528 MB/s	(transfers : 1156 - avg 2.9266 MB)
	NUMA 0 -> Disk 0	60.1317 GB	2286.9649 MB/s	(transfers : 19505 - avg 3.1569 MB)
Total transfers: 141.6908 GB
#---------------------
NNTile training time: 26.979973793029785 seconds
```

In [26]:
# Launch an external python process to train gpt2_lmhead model on TinyStories
!STARPU_LIMIT_CUDA_MEM=19000 python ../wrappers/python/examples/gpt2_lmhead_training.py \
    --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt2_default_config.json" \
    --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=fp32_fast_fp16 --nepochs=1 \
    --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin" \
    --ooc --ooc-path="/tmp/nntile_ooc" --ooc-size=10737418240 \
    --force-offload-disk-portion-parameters=0.0 --force-offload-disk-portion-gradients=1.0 \
    --force-offload-disk-portion-activations=0.0 --force-offload-disk-portion-temporaries=0.0 \
    --force-offload-disk-portion-optimizer=0.0

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='fp32_fast_fp16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=True, ooc_path='/tmp/nntile_ooc', ooc_size=10737418240, force_offload_disk_portion_parameters=0.0, force_offload_disk_portion_gradients=1.0, force_offload_disk_portion_activations=0.0, force_offload_disk_portion_temporaries=0.0, force_offload_disk_portion_optimizer=0.0, force_offload_ram_portion_parameters=0.0, force_offload_ram_portion_gradients=0.0, for

#### 6.5. Train with OOC support enabled, only 10% of inter-layer activations are offloaded to Disk. 

GPU memory is limited to 19000 MB. Only 10% of inter-layer activations are offloaded to the Disk. Training time and data transfers are reported below:
```sh
#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	5.6315 GB	219.1559 MB/s	(transfers : 1037 - avg 5.5609 MB)
	CUDA 0 -> NUMA 0	60.3561 GB	2348.8011 MB/s	(transfers : 3000 - avg 20.6015 MB)
	Disk 0 -> NUMA 0	2.5434 GB	98.9780 MB/s	(transfers : 97 - avg 26.8499 MB)
	NUMA 0 -> Disk 0	53.8887 GB	2097.1149 MB/s	(transfers : 1977 - avg 27.9120 MB)
Total transfers: 122.4197 GB
#---------------------
NNTile training time: 26.357404708862305 seconds
```

In [27]:
# Launch an external python process to train gpt2_lmhead model on TinyStories
!STARPU_LIMIT_CUDA_MEM=19000 python ../wrappers/python/examples/gpt2_lmhead_training.py \
    --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt2_default_config.json" \
    --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=fp32_fast_fp16 --nepochs=1 \
    --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin" \
    --ooc --ooc-path="/tmp/nntile_ooc" --ooc-size=10737418240 \
    --force-offload-disk-portion-parameters=0.0 --force-offload-disk-portion-gradients=0.0 \
    --force-offload-disk-portion-activations=0.1 --force-offload-disk-portion-temporaries=0.0 \
    --force-offload-disk-portion-optimizer=0.0

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='fp32_fast_fp16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=True, ooc_path='/tmp/nntile_ooc', ooc_size=10737418240, force_offload_disk_portion_parameters=0.0, force_offload_disk_portion_gradients=0.0, force_offload_disk_portion_activations=0.1, force_offload_disk_portion_temporaries=0.0, force_offload_disk_portion_optimizer=0.0, force_offload_ram_portion_parameters=0.0, force_offload_ram_portion_gradients=0.0, for

#### 6.6. Train with OOC support enabled, only 10% of temporaries are offloaded to Disk. 

GPU memory is limited to 19000 MB. Only 10% of temporaries are offloaded to the Disk. Training time and data transfers are reported below:
```sh
#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	4.7836 GB	191.7296 MB/s	(transfers : 441 - avg 11.1075 MB)
	CUDA 0 -> NUMA 0	60.5570 GB	2427.1701 MB/s	(transfers : 1918 - avg 32.3308 MB)
	Disk 0 -> NUMA 0	1.6917 GB	67.8037 MB/s	(transfers : 40 - avg 43.3070 MB)
	NUMA 0 -> Disk 0	54.0186 GB	2165.1014 MB/s	(transfers : 1442 - avg 38.3599 MB)
Total transfers: 121.0509 GB
#---------------------
NNTile training time: 25.61427140235901 seconds
```

In [28]:
# Launch an external python process to train gpt2_lmhead model on TinyStories
!STARPU_LIMIT_CUDA_MEM=19000 python ../wrappers/python/examples/gpt2_lmhead_training.py \
    --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt2_default_config.json" \
    --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=fp32_fast_fp16 --nepochs=1 \
    --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin" \
    --ooc --ooc-path="/tmp/nntile_ooc" --ooc-size=10737418240 \
    --force-offload-disk-portion-parameters=0.0 --force-offload-disk-portion-gradients=0.0 \
    --force-offload-disk-portion-activations=0.0 --force-offload-disk-portion-temporaries=0.1 \
    --force-offload-disk-portion-optimizer=0.0

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='fp32_fast_fp16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=True, ooc_path='/tmp/nntile_ooc', ooc_size=10737418240, force_offload_disk_portion_parameters=0.0, force_offload_disk_portion_gradients=0.0, force_offload_disk_portion_activations=0.0, force_offload_disk_portion_temporaries=0.1, force_offload_disk_portion_optimizer=0.0, force_offload_ram_portion_parameters=0.0, force_offload_ram_portion_gradients=0.0, for

#### 6.7.1 Training experiment with limited GPU and CPU memory with no OOC support
GPU memory is limited to 6000 MB. CPU memory is limited to 16000. Training hangs after 13861 tasks are finished:
```sh
Optimizer    (GB): 1.215
Persistent   (GB): 13.518
Temporaries  (GB): 14.698
13861 tasks finished (last 0 0x5561f728de30 on 0)....^C
```

In [2]:
# Launch an external python process to train gpt2_lmhead model on TinyStories

# This process hangs after 13861 tasks are finished, so it is commented out to prevent from automatic execution
# !STARPU_LIMIT_CUDA_MEM=6000 STARPU_LIMIT_CPU_MEM=16000 STARPU_TASK_PROGRESS=1 python ../wrappers/python/examples/gpt2_lmhead_training.py \
#     --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt2_default_config.json" \
#     --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=fp32_fast_fp16 --nepochs=1 \
#     --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin" \
#     --force-offload-ram-portion-parameters=1.0 --force-offload-ram-portion-gradients=1.0 \
#     --force-offload-ram-portion-activations=1.0 --force-offload-ram-portion-temporaries=1.0 \
#     --force-offload-ram-portion-optimizer=1.0

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='fp32_fast_fp16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=False, ooc_path='/tmp/nntile_ooc', ooc_size=1073741824, force_offload_disk_portion_parameters=0.0, force_offload_disk_portion_gradients=0.0, force_offload_disk_portion_activations=0.0, force_offload_disk_portion_temporaries=0.0, force_offload_disk_portion_optimizer=0.0, force_offload_ram_portion_parameters=1.0, force_offload_ram_portion_gradients=1.0, for

#### 6.7.2 Training experiment with limited GPU and CPU memory with OOC support for all tensors
GPU memory is limited to 6000 MB. CPU memory is limited to 16000. Training time and data transfers are reported below:
```sh
#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	1321.4303 GB	2528.0023 MB/s	(transfers : 69495 - avg 19.4711 MB)
	CUDA 0 -> NUMA 0	2373.9470 GB	4541.5510 MB/s	(transfers : 70090 - avg 34.6829 MB)
	Disk 0 -> NUMA 0	371.0725 GB	709.8915 MB/s	(transfers : 41411 - avg 9.1758 MB)
	NUMA 0 -> Disk 0	1953.4398 GB	3737.0867 MB/s	(transfers : 61513 - avg 32.5187 MB)
Total transfers: 6019.8896 GB
#---------------------
NNTile training time: 535.3222873210907 seconds
```

In [30]:
# Launch an external python process to train gpt2_lmhead model on TinyStories
!STARPU_LIMIT_CUDA_MEM=6000 STARPU_LIMIT_CPU_MEM=16000 STARPU_TASK_PROGRESS=1 python ../wrappers/python/examples/gpt2_lmhead_training.py \
    --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt2_default_config.json" \
    --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=fp32_fast_fp16 --nepochs=1 \
    --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin" \
    --ooc --ooc-path="/tmp/nntile_ooc" --ooc-size=26843545600 \
    --force-offload-disk-portion-parameters=1.0 --force-offload-disk-portion-gradients=1.0 \
    --force-offload-disk-portion-activations=1.0 --force-offload-disk-portion-temporaries=1.0 \
    --force-offload-disk-portion-optimizer=1.0

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='fp32_fast_fp16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=True, ooc_path='/tmp/nntile_ooc', ooc_size=26843545600, force_offload_disk_portion_parameters=1.0, force_offload_disk_portion_gradients=1.0, force_offload_disk_portion_activations=1.0, force_offload_disk_portion_temporaries=1.0, force_offload_disk_portion_optimizer=1.0, force_offload_ram_portion_parameters=0.0, force_offload_ram_portion_gradients=0.0, for

#### 6.7.3 Training experiment with limited GPU and CPU memory with OOC support for optimizer states
GPU memory is limited to 6000 MB. CPU memory is limited to 16000. Training does not hang, but strangely enough it does not load data from Disk to CPU RAM. Training time and data transfers are reported below:
```sh
#---------------------
Data transfer stats:
	NUMA 0 -> CUDA 0	1237.8654 GB	8065.7490 MB/s	(transfers : 68770 - avg 18.4321 MB)
	CUDA 0 -> NUMA 0	1314.8986 GB	8567.6844 MB/s	(transfers : 46997 - avg 28.6498 MB)
	Disk 0 -> NUMA 0	1.5551 GB	10.1331 MB/s	(transfers : 436 - avg 3.6524 MB)
	NUMA 0 -> Disk 0	3.3566 GB	21.8711 MB/s	(transfers : 1180 - avg 2.9128 MB)
Total transfers: 2557.6758 GB
#---------------------
NNTile training time: 157.21175980567932 seconds
```

In [31]:
# Launch an external python process to train gpt2_lmhead model on TinyStories
!STARPU_LIMIT_CUDA_MEM=6000 STARPU_LIMIT_CPU_MEM=16000 STARPU_TASK_PROGRESS=1 python ../wrappers/python/examples/gpt2_lmhead_training.py \
    --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt2_default_config.json" \
    --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=fp32_fast_fp16 --nepochs=1 \
    --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin" \
    --ooc --ooc-path="/tmp/nntile_ooc" --ooc-size=26843545600 \
    --force-offload-disk-portion-parameters=0.0 --force-offload-disk-portion-gradients=0.0 \
    --force-offload-disk-portion-activations=0.0 --force-offload-disk-portion-temporaries=0.0 \
    --force-offload-disk-portion-optimizer=1.0

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='fp32_fast_fp16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=True, ooc_path='/tmp/nntile_ooc', ooc_size=26843545600, force_offload_disk_portion_parameters=0.0, force_offload_disk_portion_gradients=0.0, force_offload_disk_portion_activations=0.0, force_offload_disk_portion_temporaries=0.0, force_offload_disk_portion_optimizer=1.0, force_offload_ram_portion_parameters=0.0, force_offload_ram_portion_gradients=0.0, for

#### 6.7.4 Training experiment with limited GPU and CPU memory with OOC support for even less CPU memory
GPU memory is limited to 6000 MB. CPU memory is limited to 10000. Training hangs after 22842 tasks are finished, presumably due to not enough memory on CPU and GPU together (which is very strange):
```sh
Activations  (GB): 11.089
Optimizer    (GB): 1.215
Persistent   (GB): 13.518
Temporaries  (GB): 14.698
22842 tasks finished (last 0 0x5609620f44b0 on 0)....^C
```

In [3]:
# Launch an external python process to train gpt2_lmhead model on TinyStories

# This process hangs after 22842 tasks are finished, so it is commented out to prevent from automatic execution
# !STARPU_LIMIT_CUDA_MEM=6000 STARPU_LIMIT_CPU_MEM=10000 STARPU_TASK_PROGRESS=1 python ../wrappers/python/examples/gpt2_lmhead_training.py \
#     --restrict="cuda" --pretrained=local --config-path="../wrappers/python/examples/gpt2_default_config.json" \
#     --save-checkpoint-path=".model/nntile_checkpoint.pt" --optimizer="adam" --lr=1e-4 --dtype=fp32_fast_fp16 --nepochs=1 \
#     --batch-size=256 --minibatch-size=8 --dataset-file="tinystories/train.bin" \
#     --ooc --ooc-path="/tmp/nntile_ooc" --ooc-size=26843545600 \
#     --force-offload-disk-portion-parameters=1.0 --force-offload-disk-portion-gradients=1.0 \
#     --force-offload-disk-portion-activations=1.0 --force-offload-disk-portion-temporaries=1.0 \
#     --force-offload-disk-portion-optimizer=1.0

Namespace(remote_model_name='openai-community/gpt2', pretrained='local', checkpoint_path='', config_path='../wrappers/python/examples/gpt2_default_config.json', save_checkpoint_path='.model/nntile_checkpoint.pt', optimizer='adam', model_path='.model', seq_len=1024, seq_len_tile=-1, batch_size=256, minibatch_size=8, minibatch_size_tile=-1, hidden_size_tile=-1, intermediate_size_tile=-1, n_head_tile=-1, dtype='fp32_fast_fp16', restrict='cuda', flash_attention=False, use_redux=False, dataset_path='.data', dataset_file='tinystories/train.bin', lr=0.0001, nepochs=1, logger=False, logger_server_addr='localhost', logger_server_port=5001, ooc=True, ooc_path='/tmp/nntile_ooc', ooc_size=26843545600, force_offload_disk_portion_parameters=1.0, force_offload_disk_portion_gradients=1.0, force_offload_disk_portion_activations=1.0, force_offload_disk_portion_temporaries=1.0, force_offload_disk_portion_optimizer=1.0, force_offload_ram_portion_parameters=0.0, force_offload_ram_portion_gradients=0.0, for